In [ ]:
%matplotlib widget

from qutip import *
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
GHz = 1e9
MHz = 1e6
ns = 1e-9
us = 1e-6
scale = GHz

wq = 6.316 * GHz * 2*np.pi / scale
wr = 4.754 * GHz * 2*np.pi / scale
delta = 340 * MHz * 2*np.pi / scale
wf = 4.752 * GHz * 2*np.pi / scale
g = 208 * MHz * 2*np.pi / scale
G = 25 * MHz * 2*np.pi / scale
Delta = wq - wr
chi = g**2/Delta - 2*g**2/(2*(Delta-delta))
T1 = 7.6 * us * scale
n_crit = Delta**2/(4*g**2)
n_d = 2.5

kf = 64 * MHz * 2*np.pi / scale
k_eff = 4*G**2/kf/(1 + (2*(wq - wf)/kf)**2)
k_eff_d = 4*G**2/kf/(1 + (2*(wr - wf)/kf)**2)
E_d = np.sqrt(n_d*(k_eff_d**2/4 + chi**2))
Gamma = g**2*k_eff/Delta**2

wd = wr
wq_prime = wq + g**2/Delta
wr_prime = wr - 2*g**2/(2*(Delta - delta))

print(f"chi/2pi= {1e3 * chi / (2*np.pi) : .3f} MHz")
print(f"wq_prime/2pi = {1e3 * wq_prime / (2*np.pi) : .3f} MHz")

In [ ]:
N = 10  # excitation number cutoff
timesteps = np.linspace(0,1,1000) # time steps

# g, e, f = basis(3, 0), basis(3, 1), basis(3, 2) # transmon basis states
# sigz_transmon = e*e.dag() - g*g.dag()
sm = tensor(sigmam(), qeye(N), qeye(N)) # qubit
sz = tensor(-sigmaz(), qeye(N), qeye(N)) # qubit
a = tensor(qeye(2), destroy(N), qeye(N)) # readout resonator
b = tensor(qeye(2), qeye(N), destroy(N)) # Purcell filter
Qr = (a + a.dag()) / 2

H_JC = (wq + g**2/Delta) * sz + (wr - g**2/(Delta-delta)) * a.dag()*a + chi * a.dag()*a * sz
H_JC_int = (wq + g**2/Delta - wd) * sz + (wr - g**2/(Delta - delta) - wd) * a.dag()*a + chi*a.dag()*a * sz
H_drive = E_d * (a + a.dag())
H_int = H_JC_int + H_drive


In [ ]:
timesteps = np.linspace(0,20,100)

c_ops = [np.sqrt(k_eff_d) * a, np.sqrt(1/T1) * sm, np.sqrt(Gamma) * sm]

psi0_g = tensor(basis(2,0), basis(N, 0), basis(N, 0))
psi0_e = tensor(basis(2,1), basis(N, 0), basis(N, 0))

result_g = mesolve(H_int, psi0_g, timesteps, c_ops)
result_e = mesolve(H_int, psi0_e, timesteps, c_ops)

Qr_0 = expect(Qr, result_g.states)
Qr_1 = expect(Qr, result_e.states)

In [ ]:
fig, ax = plt.subplots()
ax.plot(timesteps, Qr_0)
ax.plot(timesteps, Qr_1)

In [ ]:
N = 10  # excitation number cutoff
timesteps = np.linspace(0,1,1000) # time steps

psi_g = tensor(basis(3, 0), qeye(N), qeye(N))
psi_e = tensor(basis(3, 1), qeye(N), qeye(N))
psi_f = tensor(basis(3, 2), qeye(N), qeye(N)) # transmon basis states
lower_eg = psi_g*psi_e.dag()
lower_fe = psi_e*psi_f.dag()
sigz = psi_e*psi_e.dag()-psi_g*psi_g.dag() # qubit
a = tensor(qeye(3), destroy(N), qeye(N)) # readout resonator
b = tensor(qeye(3), qeye(N), destroy(N)) # Purcell filter

H_transmon= wq*psi_e*psi_e.dag() + (2*wq-delta)*psi_f*psi_f.dag()
H_resonator = wr*a.dag()*a + g*(a.dag()*lower_eg+a*lower_eg.dag()) + np.sqrt(2)*g*(a.dag()*lower_fe+a*lower_fe.dag()) 
H_filter = wf*b.dag()*b + G*(a.dag()*b+a*b.dag())
H_drive = a + a.dag()

H_exact = H_transmon + H_resonator + H_filter